**Inputs**
* ImageCollection(s) containing hazard magnitudes for each year-model pair

**Outputs**
* ImageCollection(s) containing hazard probilities for 2020 and 2060
* **You must manually create the empty ImageCollection** in the dataportal/naive-probs directory before trying to export results

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AY0e-g5Q2fz0D1eDMd0i5l7xhpXHy1J8rh8TEntsUIoyQzWLGwb4TUE1rkc

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [ ]:
hazardnames = ['dryspells']

In [5]:

WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
MODELS = ['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M']
EARLY_YEAR = 2030
LATE_YEAR = 2065

In [ ]:
def make_naiveprobs_image(year):
    result_list = ee.List([])
    for bin_idx in range(num_bins):
        model_year_list = ee.List([])
        threshold = (bin_idx + 1) * bin_width
        for y in range(year - 10, year + 10):
            for model in MODELS:
                model_year_list = model_year_list.add(magnitudes.filterMetadata('model', 'equals', model).filterMetadata('year', 'equals', year))
            threshold_list = model_year_list.mean()
        result_list = result_list.add(ee.ImageCollection(threshold_list))
    result_img = ee.ImageCollection(result_list).toBands().rename(['magnitudebin_' + str((i + 1) * bin_width) for i in range(num_bins)])
    return result_img

In [12]:
for hazardname in hazardnames:
    magnitudes = ee.ImageCollection('users/tedwongwri/dataportal/magnitudes/' + hazardname)
    for year in [EARLY_YEAR, LATE_YEAR]:
        img = make_naiveprobs_image(year)
        task = ee.batch.Export.image.toAsset(**{
          'image': img.rename(hazardname).set('model', model).set('year', year),
          'description': '{0}_{1}'.format(hazardname, str(year)),
          'assetId': 'users/tedwongwri/dataportal/rawprobs/{0}_{1}'.format(hazardname, str(year)),
          'region': WHOLE_GLOBE,
          'crs': 'EPSG:4326',
          'dimensions': '1440x720'
        })
        task.start()